In [ ]:
from pathlib import Path


import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

from skimage.morphology import binary_dilation, label
from skimage.measure import regionprops
from skimage.color import label2rgb

import tensorflow as tf

tf.test.gpu_device_name()

In [ ]:
IMAGE_DIR = Path("/Users/sylvi/topo_data/hariborings/training_data/cropped/images/")

MODEL_DIR = Path("/Volumes/shared/pyne_group/Shared/AFM_Data/Cas9_Minicircles/deep_learning/saved_models")

In [ ]:
# Load the model
model = tf.keras.models.load_model(
    MODEL_DIR / f"haribonet_multiclass_2023-10-20_14-01-15_intial_results_multiclass_cropped.h5"
)
model.summary()

In [ ]:
image_index = 0
threshold = 0.5

image = np.load(IMAGE_DIR / f"image_{image_index}.npy")
image_512 = Image.fromarray(image)
image_512 = image_512.resize((512, 512))
image_512 = np.array(image_512)
# Normalise
image_512 = image_512 - np.min(image_512)
image_512 = image_512 / np.max(image_512)

# Predict the mask
predicted_mask = model.predict(np.expand_dims(image_512, axis=0))[0]

# Gem predicted mask
predicted_gem_mask = predicted_mask[:, :, 2] > threshold
# Ring predicted mask
predicted_ring_mask = predicted_mask[:, :, 1] > threshold
# Background predicted mask
predicted_background_mask = predicted_mask[:, :, 0] > threshold

combined_predicted_mask = np.zeros_like(predicted_gem_mask).astype(np.uint8)
combined_predicted_mask[predicted_gem_mask] = 2
combined_predicted_mask[predicted_ring_mask] = 1

# Plot them side by side
fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(image_512)
ax[0].set_title("Image")
ax[1].imshow(predicted_mask)
ax[1].set_title("Predicted Mask")
ax[2].imshow(combined_predicted_mask)
ax[2].set_title("Predicted Mask")
plt.show()

In [ ]:
# Load an image